In [1]:
import torch
import torch.nn as nn
from utils.pruning import prune_model, reset_model, get_prune_percentage
from utils.training import one_epoch
import torch.nn.functional as F

from htorch import layers
from models.resnet_real import ResNet18
from models.resnet_quat import ResNet18_quat
import numpy as np
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from models.lenet_300_100 import Real, Quat

GPU = torch.device('cuda:0')
seed = 21

# class Real(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.fc1 = nn.Linear(3, 1, bias=False)

#     def forward(self, x):
#         # print(self.fc1(x))
#         return self.fc1(x).flatten()


In [2]:
model = ResNet18_quat().to(GPU)
torch.manual_seed(seed)
model.apply(reset_model)
sd1 = model.state_dict()
# print(*sd1.keys(), sep='\n')
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [3]:
torch.manual_seed(seed)
# batch_x = torch.randn(3, 3).float().to(GPU)
# batch_y = torch.randn(3, 1).flatten().to(GPU)

batch_x = torch.randn(256, 4, 224, 224).float().to(GPU)
batch_y = torch.randint(0, 1000, (256, 1)).flatten().to(GPU)
# batch_x, batch_y

In [4]:
# model.state_dict()

In [5]:
losses = []
for i in trange(5):
    loss = one_epoch(model, batch_x, batch_y, optimiser, loss_fn, GPU)
    losses.append(loss.item())
# plt.plot(losses)
# print((model(batch_x), batch_y))

  0%|          | 0/5 [00:00<?, ?it/s]/home/aritra/project/quatLT23/htorch/layers.py:591: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1691.)
  ell = torch.cholesky(cov + self.eye, upper=True)
/home/aritra/project/quatLT23/htorch/layers.py:592: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solv

In [6]:
torch.manual_seed(seed)
model = prune_model(model, 0.67)
# model.state_dict()

In [7]:
torch.manual_seed(seed)
model = model.apply(reset_model)
sd2 = model.state_dict()

In [8]:
# model1 and model2
# sd1 = list(model1.state_dict())  # unpruned
# sd2 = list(model2.state_dict())  # pruned

sdk1 = [a for a in sd1.keys() if "weight" in a]
sdk2 = [a for a in sd2.keys() if "weight" in a]

for key in sdk1:
    key2 = key+"_orig"
    if sd2.get(key2) is None:
        print(f"{key2} NOT pruned")
        continue
    if torch.allclose(sd1[key], sd2[key2]):
        # print(f"{key} and {key2} are same")
        pass
    else:
        print(f"{key} and {key2} are different!")
        break

bn1.weight_orig NOT pruned
layer1.0.bn1.weight_orig NOT pruned
layer1.0.bn2.weight_orig NOT pruned
layer1.0.bn3.weight_orig NOT pruned
layer1.0.identity_downsample.1.weight_orig NOT pruned
layer1.1.bn1.weight_orig NOT pruned
layer1.1.bn2.weight_orig NOT pruned
layer1.1.bn3.weight_orig NOT pruned
layer2.0.bn1.weight_orig NOT pruned
layer2.0.bn2.weight_orig NOT pruned
layer2.0.bn3.weight_orig NOT pruned
layer2.0.identity_downsample.1.weight_orig NOT pruned
layer2.1.bn1.weight_orig NOT pruned
layer2.1.bn2.weight_orig NOT pruned
layer2.1.bn3.weight_orig NOT pruned
layer3.0.bn1.weight_orig NOT pruned
layer3.0.bn2.weight_orig NOT pruned
layer3.0.bn3.weight_orig NOT pruned
layer3.0.identity_downsample.1.weight_orig NOT pruned
layer3.1.bn1.weight_orig NOT pruned
layer3.1.bn2.weight_orig NOT pruned
layer3.1.bn3.weight_orig NOT pruned
layer4.0.bn1.weight_orig NOT pruned
layer4.0.bn2.weight_orig NOT pruned
layer4.0.bn3.weight_orig NOT pruned
layer4.0.identity_downsample.1.weight_orig NOT pruned
l

In [9]:
model.layer1[0].bn1.state_dict().keys()
# bn_real => ['weight', 'bias', 'running_mean', 'running_var', 'num_batches_tracked']
# bn_quat => ['weight', 'bias', 'running_mean', 'running_cov', 'eye']

odict_keys(['weight', 'bias', 'eye', 'running_mean', 'running_cov'])

In [10]:
print(*sd1.keys(), sep='\n')

conv1.r_weight
conv1.i_weight
conv1.j_weight
conv1.k_weight
bn1.weight
bn1.bias
bn1.eye
bn1.running_mean
bn1.running_cov
layer1.0.conv1.r_weight
layer1.0.conv1.i_weight
layer1.0.conv1.j_weight
layer1.0.conv1.k_weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.eye
layer1.0.bn1.running_mean
layer1.0.bn1.running_cov
layer1.0.conv2.r_weight
layer1.0.conv2.i_weight
layer1.0.conv2.j_weight
layer1.0.conv2.k_weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.eye
layer1.0.bn2.running_mean
layer1.0.bn2.running_cov
layer1.0.conv3.r_weight
layer1.0.conv3.i_weight
layer1.0.conv3.j_weight
layer1.0.conv3.k_weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.bn3.eye
layer1.0.bn3.running_mean
layer1.0.bn3.running_cov
layer1.0.identity_downsample.0.r_weight
layer1.0.identity_downsample.0.i_weight
layer1.0.identity_downsample.0.j_weight
layer1.0.identity_downsample.0.k_weight
layer1.0.identity_downsample.1.weight
layer1.0.identity_downsample.1.bias
layer1.0.identity_downsample.1.eye
laye